In [2]:
import pandas as pd

# Targets

In [4]:
from Bio import SeqIO
gb_file = SeqIO.parse("../building_data/genome_modified.gb", "gb")
contigs = [ i for i in gb_file ]

In [3]:
import re

In [28]:
gb_dct = {}
for c in contigs:
    for f in c.features:
        if f.type == 'tRNA':
            s = f.qualifiers['note'][0]
            codon = re.findall('(?<=\()[A-Z]{3}(?=\))',s)
            if not codon:
                continue
            codon = codon[0]
            i = f.qualifiers['old_locus_tag'][0]
            gb_dct[codon] = i
            print(i,codon)
#             s2 = ''.join([dct[i] for i in s])
#             i = f.qualifiers['old_locus_tag'][0]
#             aa = f.qualifiers['product'][0].split('tRNA-')[1]
# #             if 'Ala' in f.qualifiers['product'][0]:
#             gb_dct[s2] = i
#             print(i,aa,s2)

BSU_tRNA_6 UGC
BSU_tRNA_7 UGA
BSU_tRNA_8 GAU
BSU_tRNA_9 UGC
BSU_tRNA_10 CAU
BSU_tRNA_11 UUC
BSU_tRNA_12 UAC
BSU_tRNA_13 UGU
BSU_tRNA_14 UUU
BSU_tRNA_15 UAG
BSU_tRNA_16 GCC
BSU_tRNA_17 UAA
BSU_tRNA_18 ACG
BSU_tRNA_19 UGG
BSU_tRNA_20 UGC
BSU_tRNA_23 GUU
BSU_tRNA_24 GGU
BSU_tRNA_25 GCC
BSU_tRNA_26 AGC
BSU_tRNA_28 UGC
BSU_tRNA_75 UUC
BSU_tRNA_78 UAC
BSU_tRNA_76 UGU
BSU_tRNA_77 GAU
BSU_tRNA_74 UUG
BSU_tRNA_45 GUU
BSU_tRNA_46 GCU
BSU_tRNA_48 UUG
BSU_tRNA_49 UUU
BSU_tRNA_50 UAG
BSU_tRNA_51 GAG
BSU_tRNA_80 UCC
BSU_tRNA_21 CAU
BSU_tRNA_22 GUC
BSU_tRNA_29 GUU
BSU_tRNA_30 GGA
BSU_tRNA_31 UUC
BSU_tRNA_32 UAC
BSU_tRNA_33 CAU
BSU_tRNA_34 GUC
BSU_tRNA_35 GAA
BSU_tRNA_36 UGU
BSU_tRNA_37 GUA
BSU_tRNA_38 CCA
BSU_tRNA_39 GUG
BSU_tRNA_40 UUG
BSU_tRNA_41 GCC
BSU_tRNA_42 GCA
BSU_tRNA_43 UAA
BSU_tRNA_44 CAA
BSU_tRNA_82 TCC
BSU_tRNA_83 GAC
BSU_tRNA_73 UUG
BSU_tRNA_84 UCU
BSU_tRNA_86 GGC
BSU_tRNA_81 CCG


In [46]:
# From https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7408541/
targets = pd.read_csv('./tRNA-mod-targets.txt',sep='\t',index_col=0, comment='#')

In [51]:
coralme_rnamod = pd.DataFrame()

In [56]:
for mod, row in targets.iterrows():
    trnas = row['target'].split(',')
    for t in trnas:
        rnamod_dct = {}
        codon = re.findall('[A-Z]{3}',t)[0]
        pos = re.findall('[0-9]{1,}',t)[0]
        if codon not in gb_dct:
            print('{} not in genbank'.format(codon))
            continue
        rnamod_dct[gb_dct[codon]] = {
            'position':pos,
            'modification':mod
        }
        coralme_rnamod = pd.concat([coralme_rnamod, pd.DataFrame.from_dict(rnamod_dct).T],join='outer')        
coralme_rnamod

GCT not in genbank
CAG not in genbank


,modification,position
BSU_tRNA_42,m1A,22
BSU_tRNA_73,m1A,22
BSU_tRNA_31,m1A,22
BSU_tRNA_44,m1A,22
BSU_tRNA_43,m1A,22
...,...,...
BSU_tRNA_33,m6A,37
BSU_tRNA_77,t6A,37
BSU_tRNA_36,t6A,37
BSU_tRNA_39,Q,34


In [71]:
coralme_rnamod[coralme_rnamod['modification'].str.contains('mo5U')]

,modification,position
BSU_tRNA_50,mo5U,34
BSU_tRNA_50,mo5U,34
BSU_tRNA_50,mo5U,34


In [57]:
# coralme_rnamod.to_csv('../building_data/post_transcriptional_modification_of_RNA.txt',sep='\t')

### Rho dependent

In [45]:
df = pd.read_csv('./TU_sequences.txt',sep='\t',index_col=0,comment='#')

In [51]:
with open('TU_sequences.fasta','w') as f:
    for tu, row in df.iterrows():
        f.write('>{}\n'.format(tu))
        f.write(row['Sequence - DNA sequence'] + '\n')

In [41]:
iJT964_TUs = pd.read_csv('./iJT964_TUs.txt',sep='\t',index_col=0,comment='#')
iJT964_TUs = iJT964_TUs[iJT964_TUs['rho_independent'] == False]

In [43]:
l = []
for tu,row in iJT964_TUs.iterrows():
    l.append(tu.split('_from_')[0].replace('_','-'))

In [44]:
df = pd.DataFrame(columns=l).T
df.index.name = 'id'
df.to_csv('../building_data/rho_dependent.txt')